In [1]:
import geopandas as gpd
import rasterio
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from skimage.feature import graycomatrix, graycoprops
import numpy as np
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)

### Load sample points and sar data


In [2]:
# Load training points
shp_path = "../sample_collection/eaton_random_sample.shp"
gdf = gpd.read_file(shp_path)

# Load SAR GeoTIFF
# input_sar_image_path  = "../SAR_Data_processing_eaton/8_subset/eaton_S1A_IW_GRDH_1SDV_2025_01_21_TC_stack_32611_clip.tif"
# glcm_raster_path="../SAR_Data_processing_eaton/9_glcm/eaton_S1A_IW_GRDH_1SDV_2025_01_21_Orb_tnr_Bdr_Cal_Spk_TC_stack_glcm.tif"
# thermal_image_path="../ThermalData/04_diff_norm/TRAD_diff_norm.tif"
# dnbr_image_path="../S2_Data_Processing/Indices/dNBR_eaton.tif"
input_sar_image_path  = "../SAR_Data_processing_eaton/8_subset/eaton_S1A_IW_GRDH_1SDV_stack_32611_clip_bilinear_10m.tif"
glcm_raster_path="input/eaton_glcm_ready.tif"
thermal_image_path="input/eaton_thermal_ready.tif"
dnbr_image_path="input/eaton_dnbr_ready.tif"
#following are the input for the model selection and training

output_file_path="output/sample/training_data_eaton_indices.csv"

# final_columns_names=["vh_post","vv_post","vh_pre","vv_pre"]
# "dTRAD","dNBR",
# "dTRAD","dNBR",
# final_columns_names=["dTRAD","dNBR","RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] # "ΔRVI",'Δvv_vh_ratio',"ΔRDFI" "RBD_VV",
# final_columns_names=["dNBR"]
# ,'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'
# RCBI: https://www.sciencedirect.com/science/article/abs/pii/S0924271619302242
final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]

# final_columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

output_sar_image_path="output/sar_image/eaton_sar_indices.tif"

add_bands=False
add_indices=True
add_glcm=False
thermal_band=False
add_dnbr=False

In [3]:
"""
Log10 of negative value is 0, thus applicanle in calculating the differnce
"""
from scipy.ndimage import generic_filter

# Open input raster
with rasterio.open(input_sar_image_path) as src:
    profile = src.profile  # Get metadata
    bands = src.read()  # Read all bands

# Define new band names (not directly stored in GeoTIFF, but useful for documentation)
# new_band_names = ["gamma_vh_post", "gamma_vv_post", "gamma_vh_pre", "gamma_vv_pre"]

# Save raster with new names
# profile.update(count=len(new_band_names)+5+glcm_bands_counts)  # Update band count if needed
# profile.update(count=len(new_band_names))  # Update band count if needed
profile.update(count=len(final_columns_names))
print(len(final_columns_names),"len(final_columns_names)")
epsilon = 1e-6
band_index=0
with rasterio.open(output_sar_image_path, "w", **profile) as dst:
    if add_bands:
        for i in range(len(bands)):
            dst.write(bands[i], i + 1)
            dst.set_band_description(i + 1, final_columns_names[i])  # Assign names
            band_index=i+1

    
    if thermal_band:
        with rasterio.open(thermal_image_path) as TRAD_dataset:
            
            # profile = src.profile  # Get metadata
            trad_band = TRAD_dataset.read()
            band_index=band_index+1
            dst.write(trad_band[0], band_index)
            dst.set_band_description(band_index, "dTRAD")
    
    if add_dnbr:
        with rasterio.open(dnbr_image_path) as dnbr_dataset:
            # profile = src.profile  # Get metadata
            dnbr_band = dnbr_dataset.read()
            band_index=band_index+1
            dst.write(dnbr_band[0], band_index)
            dst.set_band_description(band_index, "dNBR")
    
    if add_indices:
        vh_post,vv_post,vh_pre, vv_pre=bands[0],bands[1],bands[2],bands[3]

        # vh_post,vv_post,vh_pre, vv_pre=(bands[2]+bands[6])/2,(bands[3]+bands[7])/2,(bands[0]+bands[4])/2,(bands[1]+bands[5])/2

        # print(np.min(vh_post),np.max(vh_post))
        band_index=band_index+1
        RBD_VV_band = vv_post - vv_pre
        # X_min,X_max = np.min(RBD_VV_band),np.max(RBD_VV_band)
        # RBD_VV_band = (RBD_VV_band - X_min) / (X_max - X_min)
        dst.write(RBD_VV_band, band_index)
        dst.set_band_description(band_index, "RBD_VV")


        band_index=band_index+1
        RBD_VH_band = vh_post - vh_pre
        # X_min,X_max = np.min(RBD_VH_band),np.max(RBD_VH_band)
        # RBD_VH_band = (RBD_VH_band - X_min) / (X_max - X_min)
        dst.write(RBD_VH_band, band_index)
        dst.set_band_description(band_index, "RBD_VH")

        band_index=band_index+1
        # RBR_VV_band = vv_post/ (vv_pre + epsilon)
        RBR_VV_band = (np.log10(vv_post+epsilon))-(np.log10(vv_pre + epsilon)) #logarithmic ratio
        # RBR_VV_band=np.log(RBR_VV_band + epsilon) 
        # X_min,X_max = np.min(RBR_VV_band),np.max(RBR_VV_band)
        # RBR_VV_band = (RBR_VV_band - X_min) / (X_max - X_min)
        dst.write(RBR_VV_band, band_index)
        dst.set_band_description(band_index, "RBR_VV")
        
        band_index=band_index+1
        # pre_std=np.std(vv_pre) #global
        window_size = 7
        # Apply local standard deviation filter
        pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        RBR_VV_band=(np.log10(vv_post))-(np.log10(vv_pre)) #logarithmic ratio
        dod_RBR_VV= np.abs(RBR_VV_band)/pre_std_local
        dst.write(dod_RBR_VV, band_index)
        dst.set_band_description(band_index, "dod_RBR_VV")


        band_index=band_index+1
        # RBR_VH_band = vh_post / (vh_pre+epsilon)
        RBR_VH_band = (np.log10(vh_post+epsilon)) - (np.log10(vh_pre+epsilon))
        # RBR_VH_band=np.log(RBR_VH_band + epsilon) 
        # X_min,X_max = np.min(RBR_VH_band),np.max(RBR_VH_band)
        # RBR_VH_band = (RBR_VH_band - X_min) / (X_max - X_min)
        dst.write(RBR_VH_band, band_index)
        dst.set_band_description(band_index, "RBR_VH")

        band_index=band_index+1
        # pre_std=np.std(vh_pre)
        window_size = 7
        # Apply local standard deviation filter
        pre_std_local = generic_filter(vv_pre, np.std, size=window_size)
        RBR_VH_band=(np.log10(vh_post))-(np.log10(vh_pre))
        dod_RBR_VH= np.abs(RBR_VH_band)/pre_std_local
        dst.write(dod_RBR_VH, band_index)
        dst.set_band_description(band_index, "dod_RBR_VH")

        band_index=band_index+1
        # RVI_post = 4* vh_post/(vv_post+vh_post+epsilon) #4* vh/(vv+vh)
        # RVI_pre = 4* vh_pre/(vh_pre+vv_pre+epsilon) #4* vh/(vv+vh)
        RVI_post = (10*np.log10((4* vh_post) + epsilon))-(10*np.log10(vv_post+vh_post+epsilon)) #4* vh/(vv+vh)
        RVI_pre = (10*np.log10((4* vh_pre)+ epsilon))-(10*np.log10(vh_pre+vv_pre+epsilon)) #4* vh/(vv+vh)
        delta_RVI = RVI_post  - RVI_pre
        # delta_RVI=10*np.log10(delta_RVI+epsilon)
        # delta_RVI = np.log(RVI_post + epsilon) - np.log(RVI_pre + epsilon)
        # X_min, X_max = np.min(delta_RVI), np.max(delta_RVI)
        # delta_RVI = (delta_RVI - X_min) / (X_max - X_min)
        dst.write(delta_RVI, band_index)
        dst.set_band_description(band_index, "ΔRVI")

        band_index=band_index+1
        # vv_vh_ratio_post = vv_post/(vh_post+epsilon) 
        # vv_vh_ratio_pre = vv_pre/(vh_pre+epsilon)
        vv_vh_ratio_post = (10*np.log10(vv_post))-(10*np.log10(vh_post+epsilon))
        vv_vh_ratio_pre = 10*np.log10(vv_pre)-(10*np.log10(vh_pre+epsilon))
        delta_vv_vh_ratio=vv_vh_ratio_post-vv_vh_ratio_pre
        # delta_vv_vh_ratio=10*np.log10(delta_vv_vh_ratio+epsilon)
        # X_min, X_max = np.min(delta_vv_vh_ratio), np.max(delta_vv_vh_ratio)
        # delta_vv_vh_ratio = (delta_vv_vh_ratio - X_min) / (X_max - X_min)
        # delta_vv_vh_ratio=np.log(vv_vh_ratio_post+epsilon)-np.log(vv_vh_ratio_pre+epsilon)
        dst.write(delta_vv_vh_ratio, band_index)
        dst.set_band_description(band_index, "Δ_vv_vh_ratio")

        band_index=band_index+1
        RFDI_pre = (vv_pre - vh_pre) / (vv_pre + vh_pre+epsilon)
        RFDI_post = (vv_post - vh_post) / (vv_post + vh_post+epsilon)
        delta_RFDI = RFDI_post - RFDI_pre
        # X_min, X_max = np.min(delta_RFDI), np.max(delta_RFDI)
        # delta_RFDI = (delta_RFDI - X_min) / (X_max - X_min)
        dst.write(delta_RFDI, band_index)
        dst.set_band_description(band_index, "ΔRDFI")

        band_index=band_index+1
        #Radar convolution burn index
        RCBI=(vv_post-vv_pre)+(vh_post-vh_pre)
        dst.write(RCBI, band_index)
        dst.set_band_description(band_index, "RCBI")
    
    if add_glcm:
        # Open input raster
        with rasterio.open(glcm_raster_path) as glcm_dst:
            # profile = src.profile  # Get metadata
            glcm_bands = glcm_dst.read()  # Read all bands
            # print(len(glcm_bands))

        # glcm_bands_counts=len(glcm_bands)/2

        band_index=band_index+1
        with rasterio.open(glcm_raster_path) as glcm_dataset:
            glcm_bands = glcm_dataset.read()  # Read all bands
            # for i in range(16,20):
            #     difference=glcm_bands[i]-glcm_bands[i+20]
            #     dst.write(difference, band_index)
            #     band_index=band_index+1
            #     break
            for i in range(int(len(glcm_bands)/2)):
                corresponding_index,index,  = i, int(len(bands)/2)+i
                difference=glcm_bands[corresponding_index]-glcm_bands[index]
                X_min, X_max = np.min(difference), np.max(difference)
                print(X_max, X_min)
                difference = (difference - X_min) / (X_max - X_min)
                dst.write(difference+epsilon, band_index)
                band_index=band_index+1
    

10 len(final_columns_names)


In [4]:
# dataset=rasterio.open()
with rasterio.open(output_sar_image_path) as dataset:
    print(dataset.count)
    # Get coordinates of training points
    coords = [(x, y) for x, y in zip(gdf.geometry.x, gdf.geometry.y)]

    # Extract values from all bands
    values = [v for v in dataset.sample(coords)]

    # Convert to DataFrame with proper band names
    df = pd.DataFrame(values, columns=final_columns_names)

    # Merge with training sample points
    gdf = gdf.reset_index(drop=True)
    final_df = pd.concat([gdf, df], axis=1)

    # Save as CSV for ML training
    final_df.to_csv(output_file_path, index=False)

10


In [5]:
final_df.head(10)

,id,Dnbr1,dTRAD1,class,geometry,RBD_VV,RBD_VH,RBR_VV,dod_RBR_VV,RBR_VH,dod_RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
0,0.0,-0.012932,0.390035,0,POINT (402563.977 3781947.786),-0.085394,-0.004782,-0.297924,2.639620,-0.204172,1.809140,0.865775,-0.937543,-0.030446,-0.090176
1,1.0,0.257881,0.329700,1,POINT (401001.224 3788317.096),0.693183,0.149050,0.348902,0.866475,0.279478,0.694068,-0.546133,0.694244,0.053634,0.842234
2,2.0,-0.018341,0.293110,0,POINT (396536.715 3779106.757),-0.018428,-0.005861,-0.085407,1.440653,-0.197220,3.326954,-0.983460,1.118120,0.054556,-0.024289
3,3.0,0.045327,0.274815,0,POINT (394725.939 3780181.137),-0.365580,0.007311,-0.393276,3.113739,0.116031,0.918697,4.744715,-5.093075,-0.148400,-0.358269
4,4.0,0.015972,0.245232,0,POINT (403621.816 3785897.184),0.011272,0.003409,0.152463,12.430816,0.264805,21.593767,0.958312,-1.123375,-0.064929,0.014681
5,5.0,0.004455,0.215648,0,POINT (391215.942 3788010.582),-0.019720,-0.006180,-0.143719,4.522894,-0.211695,6.662564,-0.561075,0.679741,0.045137,-0.025900
6,6.0,0.189714,0.562086,1,POINT (396739.847 3785803.935),-0.029123,-0.008691,-0.131703,2.127330,-0.068483,1.106189,0.401222,-0.632206,-0.067478,-0.037814
7,7.0,0.355512,0.508758,1,POINT (396734.55 3787246.972),3.178835,0.006219,0.709154,2.586889,0.017776,0.064844,-6.315142,6.913790,0.247803,3.185054
8,8.0,0.022919,0.337096,0,POINT (400108.795 3779769.036),0.170491,0.004143,0.428766,16.672318,0.055241,2.148048,-3.111945,3.735279,0.237127,0.174634
9,9.0,0.018437,0.305566,0,POINT (392028.617 3779193.377),-0.014485,-0.023040,-0.032378,0.173885,-0.284755,1.529299,-2.139862,2.523771,0.149383,-0.037525


In [6]:
df.agg(['min', 'max'])

,RBD_VV,RBD_VH,RBR_VV,dod_RBR_VV,RBR_VH,dod_RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
min,-1.953554,-0.422484,-0.756728,0.000416,-1.267929,0.000264,-14.016992,-9.988633,-0.875591,-2.064492
max,5.741815,0.638357,1.805561,216.663483,1.513628,185.473175,7.720948,15.615835,1.214909,5.602402


In [7]:
# 	dTRAD	dNBR	RBD_VV	RBD_VH	RBR_VV	dod_RBR_VV	RBR_VH	dod_RBR_VH	ΔRVI	Δvv_vh_ratio	ΔRDFI	RCBI
# min	0.057610	-0.248237	-1.953554	-0.422484	-0.756728	0.000416	-1.267929	0.000264	-14.016992	-9.988633	-0.875591	-2.064492
# max	0.958609	0.739910	5.741815	0.638357	1.805561	216.663483	1.513628	185.473175	7.720948	15.615835	1.214909	5.602402